In [49]:
def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, gap=-1):
    m, n = len(seq1), len(seq2)
    score = [[0 for j in range(n+1)] for i in range(m+1)]
    traceback = [["" for j in range(n+1)] for i in range(m+1)]
    
    for i in range(m+1):
        score[i][0] = gap * i
    for j in range(n+1):
        score[0][j] = gap * j
        
    traceback[0][0] = "END"
    
    for i in range(1, m+1):
        for j in range(1, n+1):
            match_score = score[i-1][j-1] + (match if seq1[i-1] == seq2[j-1] else mismatch)
            delete = score[i-1][j] + gap
            insert = score[i][j-1] + gap
            score[i][j] = max(match_score, delete, insert)
            
            if score[i][j] == delete:
                traceback[i][j] += "↑ "  # UP
            if score[i][j] == insert:
                traceback[i][j] += "← "  # LEFT
            if score[i][j] == match_score:
                traceback[i][j] += "↖ "  # DIAG
            
    # Traceback to get the alignment
    alignment1, alignment2 = "", ""
    i, j = m, n
    while traceback[i][j] != "END":
        traceback_dir = traceback[i][j].split(" ")[0]
        if traceback_dir == "↖":
            alignment1 += seq1[i-1]
            alignment2 += seq2[j-1]
            i -= 1
            j -= 1
        elif traceback_dir == "↑":
            alignment1 += seq1[j-1]
            alignment2 += '-'
            i -= 1
        else:  # traceback_dir == "←"
            alignment1 += '-'
            alignment2 += seq2[i-1]
            j -= 1

    alignment1 = alignment1[::-1]
    alignment2 = alignment2[::-1]
    return score, traceback, alignment1, alignment2
   
                
    
    

In [52]:
import pandas as pd
from tabulate import tabulate
import sys

argv = sys.argv

if __name__ == "__main__":
    seq1, seq2 = argv[1], argv[2]
    score, traceback, alignment1, alignment2 = needleman_wunsch(seq1, seq2)

    # Create pandas DataFrame from score and traceback matrices
    score_df = pd.DataFrame(score, index=[''] + list(seq1), columns=[''] + list(seq2))
    traceback_df = pd.DataFrame(traceback, index=[''] + list(seq1), columns=[''] + list(seq2))

    print("Score Matrix:\n")
    print(tabulate(score_df, headers='keys', tablefmt='psql'))

    print("\nTraceback Matrix:\n")
    print(tabulate(traceback_df, headers='keys', tablefmt='psql'))

    print("\nAlignment:")
    print("Sequence 1: ", alignment1)
    print("Sequence 2: ", alignment2)


Score Matrix:

+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|    |     |   - |   - |   s |   t |   d |   i |   n |   = |   9 |   0 |   0 |   3 |
|----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----|
|    |   0 |  -1 |  -2 |  -3 |  -4 |  -5 |  -6 |  -7 |  -8 |  -9 | -10 | -11 | -12 |
| -  |  -1 |   1 |   0 |  -1 |  -2 |  -3 |  -4 |  -5 |  -6 |  -7 |  -8 |  -9 | -10 |
| -  |  -2 |   0 |   2 |   1 |   0 |  -1 |  -2 |  -3 |  -4 |  -5 |  -6 |  -7 |  -8 |
| i  |  -3 |  -1 |   1 |   1 |   0 |  -1 |   0 |  -1 |  -2 |  -3 |  -4 |  -5 |  -6 |
| p  |  -4 |  -2 |   0 |   0 |   0 |  -1 |  -1 |  -1 |  -2 |  -3 |  -4 |  -5 |  -6 |
| =  |  -5 |  -3 |  -1 |  -1 |  -1 |  -1 |  -2 |  -2 |   0 |  -1 |  -2 |  -3 |  -4 |
| 1  |  -6 |  -4 |  -2 |  -2 |  -2 |  -2 |  -2 |  -3 |  -1 |  -1 |  -2 |  -3 |  -4 |
| 2  |  -7 |  -5 |  -3 |  -3 |  -3 |  -3 |  -3 |  -3 |  -2 |  -2 |  -2 |  -3 |  -4 |
| 7  |  -8 |  -6 |  -4 |  -4 |  -4 |  -4 |  -4 |  